In [1]:
#import libraries
import numpy as np
import pandas as pd


import warnings
warnings.filterwarnings('ignore')

In [2]:
#import dataset
data = pd.read_csv('C:\\Users\\pkmra_fk94jjw\\Downloads\\judge-1377884607_tweet_product_company.csv',encoding = 'ISO-8859-1')
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [3]:
data.shape

(9093, 3)

In [4]:
pd.set_option('display.max_colwidth', None)
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion


In [5]:
#setting text and label from the columns
message = data[['is_there_an_emotion_directed_at_a_brand_or_product','tweet_text']]
message.columns = ["label", "text"]
message.head()

,label,text
0,Negative emotion,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW."
1,Positive emotion,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW"
2,Positive emotion,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.
3,Negative emotion,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw
4,Positive emotion,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)"


In [6]:
message.label.value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: label, dtype: int64

In [7]:
#mapping the responses to specific number
message['label'] = message['label'].map({'No emotion toward brand or product':2,'Positive emotion':3,'Negative emotion':1,"I can't tell":0})

In [8]:
message.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   9093 non-null   int64 
 1   text    9092 non-null   object
dtypes: int64(1), object(1)
memory usage: 142.2+ KB


In [9]:
#data preprocessing
message=message.dropna()

In [10]:
from keras.preprocessing import text
tokenizer = text.Tokenizer() 
tokenizer.fit_on_texts(list(message['text']))
tokenized_texts = tokenizer.texts_to_sequences(message['text'])

In [11]:
from keras.utils import pad_sequences
X = pad_sequences(tokenized_texts, maxlen=100)

In [12]:
#splitting dataset to train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, message['label'].values, test_size=0.2)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding,SimpleRNN,Dropout

In [14]:
#conduting a sequential model
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim = 128,input_length=100))
model.add(SimpleRNN(10))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  

In [15]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1298944   
                                                                 
 simple_rnn (SimpleRNN)      (None, 10)                1390      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,300,935
Trainable params: 1,300,935
Non-

In [17]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1)    

Epoch 1/10
205/205 [==============================] - 10s 40ms/step - loss: -12.3644 - accuracy: 0.0622 - val_loss: -37.4492 - val_accuracy: 0.0646
Epoch 2/10
205/205 [==============================] - 8s 38ms/step - loss: -97.5103 - accuracy: 0.0623 - val_loss: -169.0179 - val_accuracy: 0.0646
Epoch 3/10
205/205 [==============================] - 8s 38ms/step - loss: -285.3864 - accuracy: 0.0628 - val_loss: -389.0949 - val_accuracy: 0.0646
Epoch 4/10
205/205 [==============================] - 8s 38ms/step - loss: -564.9672 - accuracy: 0.0628 - val_loss: -689.6674 - val_accuracy: 0.0646
Epoch 5/10
205/205 [==============================] - 8s 40ms/step - loss: -908.6682 - accuracy: 0.0629 - val_loss: -1055.5841 - val_accuracy: 0.0646
Epoch 6/10
205/205 [==============================] - 8s 38ms/step - loss: -1340.4274 - accuracy: 0.0633 - val_loss: -1496.9224 - val_accuracy: 0.0646
Epoch 7/10
205/205 [==============================] - 8s 39ms/step - loss: -1839.7632 - accuracy: 0.0633 

In [18]:
y_pred = model.predict(X_test)

57/57 [==============================] - 1s 7ms/step


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.061572292468389224